In [1]:
import librosa
import os # библиотека для работы с файлами
import matplotlib.pyplot as plt
import librosa.display as ld
import numpy as np
import pandas as pd
import random

## Предобработка всех файлов директории, создание датафрейма

### Извлечение характеристик из файлов

In [2]:
n_fft = 8195
sr = 22050
hop_length = n_fft // 2

def spectrum_feature(signal):
    ft = np.abs(librosa.stft(signal[:n_fft], n_fft=n_fft, hop_length=hop_length))
    mean_ft = np.mean(ft, axis=1)  # Усредняем по всем каналам

    return np.mean(mean_ft), np.var(mean_ft)

def db_spectrum_feature(signal):
    X = librosa.stft(signal)
    s = librosa.amplitude_to_db(abs(X))

    return np.mean(s), np.var(s)

def melspectrum_feature(signal):
    mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc = 40, hop_length=512)

    return np.mean(mfccs), np.var(mfccs)

def db_melspectrum_feature(signal):
    melspectrum = librosa.feature.melspectrogram(y=signal, sr=sr, hop_length=512, n_mels=40)
    melspectrum_db = librosa.power_to_db(melspectrum, ref=np.max)  # Преобразуем в децибелы

    return np.mean(melspectrum_db), np.var(melspectrum_db)

def spectral_cent_feature(signal):
    cent = librosa.feature.spectral_centroid(y=signal, sr=sr)

    return np.mean(cent), np.var(cent)

def spectral_rolloff_feature(signal):
    rolloff = librosa.feature.spectral_rolloff(y=signal, sr=sr)

    return np.mean(rolloff), np.var(rolloff)

def zero_crossing_feature(signal):
    zrate=librosa.feature.zero_crossing_rate(signal)

    return np.mean(zrate), np.var(zrate)

### Извлечение MFCC по группам

In [12]:
# Функция для вычисления мел-кепстральных коэффициентов
def compute_mfcc(audio, sr=22050, n_mfcc=13):
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    return mfccs

# Функция для разбиения аудио на фрагменты по 20 секунд
def split_audio(audio, sr=22050, fragment_length=20):
    fragment_samples = fragment_length * sr
    num_fragments = len(audio) // fragment_samples
    fragments = [audio[i*fragment_samples:(i+1)*fragment_samples] for i in range(num_fragments)]
    return fragments

# Функция для разбиения мел-кепстральных коэффициентов на группы по 20 элементов
def split_mfcc(mfccs, group_size=20):
    num_frames = mfccs.shape[1]
    num_groups = num_frames // group_size
    mfcc_groups = [mfccs[:, i*group_size:(i+1)*group_size] for i in range(num_groups)]
    return mfcc_groups

# Функция для обработки отдельного аудиофайла
def process_audio(file_path, max_duration, fragment_length=20, group_size=20):
    try:
        audio, sr = librosa.load(file_path, sr=None, duration=max_duration)
        audio_length = librosa.get_duration(y=audio, sr=sr)
        if audio_length < fragment_length:
            return None
        fragments = split_audio(audio, sr, fragment_length)
        data = []
        filename = os.path.splitext(os.path.basename(file_path))[0]
        for i, fragment in enumerate(fragments):
            mfccs = compute_mfcc(fragment, sr)
            mfcc_groups = split_mfcc(mfccs, group_size)
            fragment_data = {'filename': f"{filename}_{i+1}"}
            for j, group in enumerate(mfcc_groups):
                mean = np.mean(group, axis=1)
                std_dev = np.std(group, axis=1)
                for k in range(mean.shape[0]):
                    fragment_data[f'MFCC_Mean_{(j*group_size)+k+1}'] = mean[k]
                    fragment_data[f'MFCC_StdDev_{(j*group_size)+k+1}'] = std_dev[k]
            data.append(fragment_data)
        return data
    except Exception as e:
        print(f"Error processing audio {file_path}: {e}")
        return None



# Функция для обработки всех аудиофайлов в директории
def process_directory(directory_path, fragment_length=20, group_size=20, max_duration=60):
    data = []
    for filename in os.listdir(directory_path):
        if filename.endswith('.mp3'):
            file_path = os.path.join(directory_path, filename)
            result = process_audio(file_path, max_duration, fragment_length, group_size)
            if result is not None:
                data.extend(result)
    return data

# Путь к директории с аудиофайлами
directory_path = 'songs'
# Обработка всех аудиофайлов в директории
data = process_directory(directory_path)

csv_path = 'songs_features.csv'

try:
    with open(csv_path, 'w') as file:
        file.truncate(0)
    print(f"Файл '{csv_path}' очищен успешно.")
except Exception as e:
    print(f"Ошибка при очистке файла '{csv_path}': {e}")

# Создание датафрейма из полученных данных
df = pd.DataFrame(data)

# Вывод датафрейма
print(df)
csv_path = 'songs_features.csv'
df.to_csv(csv_path, index=False)

Файл 'songs_features.csv' очищен успешно.
                                             filename  MFCC_Mean_1  \
0         7 Seconds -- Youssou N'Dour, Neneh Cherry_1  -551.628662   
1         7 Seconds -- Youssou N'Dour, Neneh Cherry_2  -140.451599   
2         7 Seconds -- Youssou N'Dour, Neneh Cherry_3  -170.317963   
3                                a-ha -- Take On Me_1  -540.525757   
4                                a-ha -- Take On Me_2  -126.084801   
5                                a-ha -- Take On Me_3   -64.118607   
6                       ABBA -- Money, Money, Money_1  -543.826843   
7                       ABBA -- Money, Money, Money_2  -212.842972   
8                       ABBA -- Money, Money, Money_3  -137.955460   
9                   ABBA -- The Winner Takes It All_1  -698.643555   
10                  ABBA -- The Winner Takes It All_2  -212.278900   
11                  ABBA -- The Winner Takes It All_3  -301.265747   
12                          All The Things She S

### Обработка аудиофайлов целиком

In [9]:
csv_path = 'songs_features.csv'

try:
    with open(csv_path, 'w') as file:
        file.truncate(0)
    print(f"Файл '{csv_path}' очищен успешно.")
except Exception as e:
    print(f"Ошибка при очистке файла '{csv_path}': {e}")


# Путь к папке с аудиокомпозициями
songs_dir = 'songs/'

# Список имен файлов в папке songs
file_names = os.listdir(songs_dir)

# Создание пустого DataFrame с колонкой "filename"
df = pd.DataFrame({'filename': file_names})

# Добавление признаков в DataFrame
for func_name, func in [('spectrum_feature', spectrum_feature),
                        ('db_spectrum_feature', db_spectrum_feature),
                        ('melspectrum_feature', melspectrum_feature),
                        ('db_melspectrum_feature', db_melspectrum_feature),
                        ('spectral_cent_feature', spectral_cent_feature),
                        ('spectral_rolloff_feature', spectral_rolloff_feature),
                        ('zero_crossing_feature', zero_crossing_feature)]:
    # Применение функции к каждому файлу и добавление результатов в DataFrame
    mean_var = df['filename'].apply(lambda x: pd.Series(func(librosa.load(songs_dir + x, sr=22050)[0]))) # решить с каналом
    df[[f'{func_name}_mean', f'{func_name}_var']] = mean_var

# Вывод DataFrame
print(df)
df.to_csv(csv_path, index=False)

Файл 'songs_features.csv' очищен успешно.
                                             filename  spectrum_feature_mean  \
0       7 Seconds -- Youssou N'Dour, Neneh Cherry.mp3           1.415024e-03   
1                              a-ha -- Take On Me.mp3           8.429580e-04   
2                     ABBA -- Money, Money, Money.mp3           7.433493e-20   
3                 ABBA -- The Winner Takes It All.mp3           8.187937e-06   
4                         All The Things She Said.mp3           6.735864e-02   
5                         Animals — Martin Garrix.mp3           1.317450e+00   
6              Another One Bites The Dust — Queen.mp3           2.534112e+00   
7                                       Apologize.mp3           5.115469e-01   
8                                       Appletree.mp3           1.043127e+00   
9   B.o.B, Hayley Williams of Paramore -- Airplane...           1.066156e-01   
10                   B.o.B, Jessie J -- Price Tag.mp3           1.173418e+00  

### Обработка дискретизированного аудиофайла

In [6]:
n_fft = 8195
sr = 22050
hop_length = n_fft // 2
segment_duration = 20  # Длительность каждого сегмента в секундах
segment_samples = segment_duration * sr  # Количество отсчетов в каждом сегменте

def segment_audio(signal, sr, segment_duration):
    # Разбиваем аудиозапись на сегменты по segment_duration секунд
    segments = []
    for start_sample in range(0, len(signal), segment_duration * sr):
        segment = signal[start_sample:start_sample + segment_duration * sr]
        # Если сегмент короче ожидаемой длительности, дополним его нулями
        if len(segment) < segment_duration * sr:
            segment = np.concatenate([segment, np.zeros(segment_duration * sr - len(segment))])
        segments.append(segment)
    return segments

def extract_features(signal):
    features = {}
    features['spectrum_mean'], features['spectrum_var'] = spectrum_feature(signal)
    features['db_spectrum_mean'], features['db_spectrum_var'] = db_spectrum_feature(signal)
    features['melspectrum_mean'], features['melspectrum_var'] = melspectrum_feature(signal)
    features['db_melspectrum_mean'], features['db_melspectrum_var'] = db_melspectrum_feature(signal)
    features['spectral_cent_mean'], features['spectral_cent_var'] = spectral_cent_feature(signal)
    features['spectral_rolloff_mean'], features['spectral_rolloff_var'] = spectral_rolloff_feature(signal)
    features['zero_crossing_mean'], features['zero_crossing_var'] = zero_crossing_feature(signal)
    return features


csv_path = 'songs_features.csv'

try:
    with open(csv_path, 'w') as file:
        file.truncate(0)
    print(f"Файл '{csv_path}' очищен успешно.")
except Exception as e:
    print(f"Ошибка при очистке файла '{csv_path}': {e}")

# Путь к папке с аудиокомпозициями
songs_dir = 'songs/'

# Список имен файлов в папке songs
file_names = os.listdir(songs_dir)

# Создание пустого списка для хранения словарей признаков
segments_data = []

# Добавление признаков в DataFrame для каждого сегмента каждой композиции

for filename in file_names:
    file_path = os.path.join(songs_dir, filename)
    signal, sr = librosa.load(file_path, sr=sr)
    segments = segment_audio(signal, sr, segment_duration)
    for i, segment in enumerate(segments):
        # Обрезаем тишину в начале и в конце сегмента
        segment, _ = librosa.effects.trim(segment) # эта строка убирается, если тишину необходимо оставить
        features = extract_features(segment)
        segment_data = {'filename': f"{filename[:-4]}_{i+1}", **features}
        segments_data.append(segment_data)

# Создание DataFrame из списка словарей
df = pd.DataFrame(segments_data)

# Вывод DataFrame
print(df)
df.to_csv(csv_path, index=False)


Файл 'songs_features.csv' очищен успешно.
                                        filename  spectrum_mean  spectrum_var  \
0    7 Seconds -- Youssou N'Dour, Neneh Cherry_1       0.009078      0.001058   
1    7 Seconds -- Youssou N'Dour, Neneh Cherry_2       2.130328     40.773502   
2    7 Seconds -- Youssou N'Dour, Neneh Cherry_3       2.368254     52.991505   
3    7 Seconds -- Youssou N'Dour, Neneh Cherry_4       2.133529     49.443848   
4    7 Seconds -- Youssou N'Dour, Neneh Cherry_5       3.533919     61.632301   
..                                           ...            ...           ...   
169              Bag Raiders -- Shooting stars_8       5.743456    126.091370   
170              Bag Raiders -- Shooting stars_9       7.146947     78.124115   
171             Bag Raiders -- Shooting stars_10       8.448624    131.645828   
172             Bag Raiders -- Shooting stars_11       6.004018    144.951797   
173             Bag Raiders -- Shooting stars_12       1.797930    

## Тестирование

In [10]:
df = pd.read_csv('songs_features.csv')
filename_new = 'Lady_Gaga_-_Bad_Romance_Original_47111597.mp3'
signal = librosa.load(filename_new, sr=22050)[0]
spectrum_mean, spectrum_var = spectrum_feature(signal)
db_spectrum_mean, db_spectrum_var = db_spectrum_feature(signal)
melspectrum_mean, melspectrum_var = melspectrum_feature(signal)
db_melspectrum_mean, db_melspectrum_var = db_melspectrum_feature(signal)
spectral_cent_mean, spectral_cent_var = spectral_cent_feature(signal)
spectral_rolloff_mean, spectral_rolloff_var = spectral_rolloff_feature(signal)
zero_crossing_mean, zero_crossing_var = zero_crossing_feature(signal)

new_features = [spectrum_mean, spectrum_var, db_spectrum_mean, db_spectrum_var, melspectrum_mean, melspectrum_var, db_melspectrum_mean, db_melspectrum_var, spectral_cent_mean, spectral_cent_var, spectral_rolloff_mean, spectral_rolloff_var, zero_crossing_mean, zero_crossing_var]
new_features = np.array(new_features).reshape(1, -1)



X = df.drop(columns=['filename'])
print(X)
print('Features:')
print(new_features)

    spectrum_feature_mean  spectrum_feature_var  db_spectrum_feature_mean  \
0            1.415024e-03          5.448380e-06                 -8.287863   
1            8.429581e-04          8.048017e-06                 -3.834793   
2            7.433493e-20          1.894931e-39                 -7.576152   
3            8.187937e-06          3.205970e-11                 -6.590091   
4            6.735864e-02          2.151134e-01                 -2.964818   
5            1.317450e+00          2.668646e+00                 -5.605087   
6            2.534112e+00          8.626266e+01                 -9.704225   
7            5.115468e-01          5.532359e+00                 -6.469716   
8            1.043127e+00          3.236451e+01                -14.959120   
9            1.066156e-01          1.368918e-01                 -2.647291   
10           1.173418e+00          1.380086e+01                 -1.725452   
11           1.118220e-01          5.227377e-02                 -5.999099   

### Поиск по дистанциям

In [11]:
# Вычислите расстояния между наборами признаков
distances = np.linalg.norm(X.values - new_features, axis=1)

# Найдите индексы строк с минимальными расстояниями (включая самый близкий)
closest_indices = np.argsort(distances)[:11]  # Выбираем первые четыре индекса с наименьшими расстояниями

# Получите названия файлов из найденных строк
closest_files = df.iloc[closest_indices]['filename']

# Получите признаки из найденных строк
closest_features = X.iloc[closest_indices].values

print("Признаки новой композиции:")
print(new_features)

print("\nДругие похожие файлы и их признаки:")
for i in range(len(closest_indices)):
    print(f"\nФайл {i+1}: {closest_files.iloc[i]}")
    print(closest_features[i])
    print("Расстояние:", distances[closest_indices[i]])


Признаки новой композиции:
[[ 2.11948013e+00  4.30794830e+01  2.96823800e-01  1.56681458e+02
   2.79485059e+00  3.84395752e+02 -3.39155273e+01  1.24760658e+02
   2.60069017e+03  3.18842475e+05  5.71951640e+03  1.33462482e+06
   9.97256123e-02  1.96268942e-03]]

Другие похожие файлы и их признаки:

Файл 1: Bad Romance.mp3
[ 1.25782840e+00  1.30669450e+01  1.59998520e-01  1.56046080e+02
  2.68393300e+00  3.81056060e+02 -3.37696340e+01  1.22695940e+02
  2.65413866e+03  3.28034892e+05  5.84806078e+03  1.32579123e+06
  1.02772814e-01  2.13159953e-03]
Расстояние: 12749.627725956465

Файл 2: a-ha -- Take On Me.mp3
[ 8.42958050e-04  8.04801700e-06 -3.83479300e+00  1.94879550e+02
  6.73671700e-01  5.73957100e+02 -3.27405400e+01  1.50708020e+02
  3.06656841e+03  4.99709497e+05  6.38664478e+03  1.30746581e+06
  1.57460483e-01  4.60520767e-03]
Расстояние: 182896.6728782581

Файл 3: Bad Bad Boys.mp3
[ 1.11821980e-01  5.22737700e-02 -5.99909900e+00  1.70628970e+02
 -6.54116750e-01  5.98956600e+02 -3

### Поиск по сумме дистанций фрагментов (только для дискретизованных)

In [ ]:
# Создайте словарь для хранения сумм расстояний по композициям
sum_distances = {}

# Проход по каждой строке DataFrame X (признаки каждого фрагмента)
for index, row in df.iterrows():
    # Преобразуйте индекс в строку и получите название композиции из индекса строки
    composition_name = row['filename'].split('_')[0]
    
    # Если композиция еще не добавлена в словарь, добавьте ее
    if composition_name not in sum_distances:
        sum_distances[composition_name] = 0
    
    # Вычислите расстояние между текущим фрагментом и новой композицией
    distance = np.linalg.norm(X.values - new_features)
    
    # Добавьте расстояние к сумме расстояний для данной композиции
    sum_distances[composition_name] += distance

# Преобразуйте словарь в DataFrame для удобства обработки
sum_distances_df = pd.DataFrame(list(sum_distances.items()), columns=['Composition', 'Sum Distance'])

# Найдите наиболее похожие композиции на основе сумм расстояний
closest_compositions = sum_distances_df.nsmallest(11, 'Sum Distance')

# Выведите результаты
print("Наиболее похожие композиции:")
for index, row in closest_compositions.iterrows():
    print(f"{row['Composition']}, Сумма расстояний: {row['Sum Distance']}")


Наиболее похожие композиции:
B.o.B, Hayley Williams of Paramore -- Airplanes (feat. Hayley Williams of Paramore), Сумма расстояний: 256515496.88572505
ABBA -- Money, Money, Money, Сумма расстояний: 285017218.7619167
Apologize, Сумма расстояний: 285017218.7619167
All The Things She Said, Сумма расстояний: 313518940.6381084
Another One Bites The Dust — Queen, Сумма расстояний: 313518940.6381084
Bad Bad Boys, Сумма расстояний: 313518940.6381084
a-ha -- Take On Me, Сумма расстояний: 342020662.51430005
B.o.B, Jessie J -- Price Tag, Сумма расстояний: 342020662.51430005
Bag Raiders -- Shooting stars, Сумма расстояний: 342020662.51430005
Appletree, Сумма расстояний: 399024106.2666834
ABBA -- The Winner Takes It All, Сумма расстояний: 427525828.1428751


### Поиск для MFCC

### Загрузка файла

In [20]:
df = pd.read_csv('songs_features.csv')
filename_new = 'test_data/Lady_Gaga_-_Bad_Romance_Original_47111597.mp3'
result = process_audio(filename_new, 20, fragment_length=20, group_size=20)
result = list(result[0].values())[1:]
new_features = np.array(result).reshape(1, -1)

X = df.drop(columns=['filename'])
print('Features:')
print(new_features)

Features:
[[-348.66214   188.23857    46.69595  ...   13.526917   -8.243434
     8.382611]]


In [21]:
# Вычислите расстояния между наборами признаков
distances = np.linalg.norm(X.values - new_features, axis=1)

# Найдите индексы строк с минимальными расстояниями (включая самый близкий)
closest_indices = np.argsort(distances)[:11]  # Выбираем первые четыре индекса с наименьшими расстояниями

# Получите названия файлов из найденных строк
closest_files = df.iloc[closest_indices]['filename']

# Получите признаки из найденных строк
closest_features = X.iloc[closest_indices].values

print("Признаки новой композиции:")
print(new_features)

print("\nДругие похожие файлы и их признаки:")
for i in range(len(closest_indices)):
    print(f"\nФайл {i+1}: {closest_files.iloc[i]}")
    print(closest_features[i])
    print("Расстояние:", distances[closest_indices[i]])

Признаки новой композиции:
[[-348.66214   188.23857    46.69595  ...   13.526917   -8.243434
     8.382611]]

Другие похожие файлы и их признаки:

Файл 1: Bad Romance_1
[-5.2560530e+02  6.1035156e-05  0.0000000e+00 ...  7.4524070e+00
 -1.0598659e+01  6.6915894e+00]
Расстояние: 509.23697720421603

Файл 2: B.o.B, Jessie J -- Price Tag_3
[-92.25037    25.837334   83.925     ...   6.299882   -4.8094645
   4.7632427]
Расстояние: 980.5564010382245

Файл 3: a-ha -- Take On Me_3
[-64.11861    16.832825   87.00343   ...   8.447357   -1.0431896
   5.5425186]
Расстояние: 1028.801573936934

Файл 4: Bad Romance_3
[-80.02391    37.33092   104.14042   ...   6.2745314  -1.7082291
   6.9051437]
Расстояние: 1038.9317342723064

Файл 5: a-ha -- Take On Me_2
[-126.0848      27.13207    105.16463   ...    8.085495    -6.4698553
    5.8936   ]
Расстояние: 1060.8070067363572

Файл 6: Bad Romance_2
[-164.10403     29.969702   106.84413   ...    6.0424314    9.313425
    5.54977  ]
Расстояние: 1082.254790022362

In [18]:
# Создайте словарь для хранения сумм расстояний по композициям
sum_distances = {}

# Проход по каждой строке DataFrame X (признаки каждого фрагмента)
for index, row in df.iterrows():
    # Преобразуйте индекс в строку и получите название композиции из индекса строки
    composition_name = row['filename'].split('_')[0]
    
    # Если композиция еще не добавлена в словарь, добавьте ее
    if composition_name not in sum_distances:
        sum_distances[composition_name] = 0
    
    # Вычислите евклидово расстояние между текущим фрагментом и новой композицией
    distance = np.linalg.norm(row.values[1:] - new_features)
    
    # Добавьте расстояние к сумме расстояний для данной композиции
    sum_distances[composition_name] += distance

# Преобразуйте словарь в DataFrame для удобства обработки
sum_distances_df = pd.DataFrame(list(sum_distances.items()), columns=['Composition', 'Sum Distance'])

closest_compositions = sum_distances_df.nsmallest(11, 'Sum Distance')

print("Наиболее похожие композиции:")
for index, row in closest_compositions.iterrows():
    print(f"{row['Composition']}, Сумма расстояний: {row['Sum Distance']}")


Наиболее похожие композиции:
Bad Romance, Сумма расстояний: 2630.4235014988844
B.o.B, Jessie J -- Price Tag, Сумма расстояний: 3515.387924197815
a-ha -- Take On Me, Сумма расстояний: 3874.840993673385
Bad Bad Boys, Сумма расстояний: 3951.1581485238803
All The Things She Said, Сумма расстояний: 4736.53601126331
B.o.B, Hayley Williams of Paramore -- Airplanes (feat. Hayley Williams of Paramore), Сумма расстояний: 4799.102430586729
7 Seconds -- Youssou N'Dour, Neneh Cherry, Сумма расстояний: 4883.508073258614
Animals — Martin Garrix, Сумма расстояний: 5028.329422064423
ABBA -- Money, Money, Money, Сумма расстояний: 5170.1217191975065
Bag Raiders -- Shooting stars, Сумма расстояний: 5198.393368656863
Apologize, Сумма расстояний: 5801.0897411787
